### Cell mask segmentation

with Patrick's 2D fixed image pFAK and Zyxin

Liya Ding, 2024.11

In [3]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

from skimage.morphology import binary_opening, binary_closing, binary_dilation
from skimage.morphology import disk

from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization
from scipy import ndimage as ndi

In [4]:
# import my functions
from zyxin_pfak_cell_mask_seg import zyxin_pfak_cell_mask_seg

In [5]:
# build the colormap with iterative tab10
tab10 = cm.get_cmap('tab10', 10)
for i in range(10):
    tab10.colors = np.concatenate([tab10.colors,tab10.colors],axis=0)    
tab10.colors = np.concatenate([np.zeros([1,4]),tab10.colors],axis=0)

newmap = cm.get_cmap('tab10', 4000+1)
newmap.colors = tab10.colors[0:4000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

In [6]:
# flag for some optinal parts
flag_plot = True
flag_plot_save = True
flag_seg_save = True
flag_csv_save = True
flag_run_all = True
flag_panel_plot = True

In [7]:
image_2D_2CH_dir =  '/mnt/d/lding/zyxin/data/240423_fixed_EGFP_zyxin_pFAK/Y27_images'
output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/Y27_pfak'
method_ID = 'cellmask_redo'

csv_output_dir = os.path.join(output_dir, method_ID+'_csv')
plot_output_dir = os.path.join(output_dir, method_ID+'_plot')
seg_output_dir = os.path.join(output_dir, method_ID+'_seg')

pixel_size = 0.0706
time_point = 0

if not os.path.isdir(csv_output_dir):     os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):     os.makedirs(seg_output_dir)

In [8]:
### for each movie, make sub-folders for different output for easier viewing in imagej
movie_mask_output_dir = os.path.join(seg_output_dir,  'mask')
movie_seg_output_dir = os.path.join(seg_output_dir,  'seg')
movie_label_output_dir = os.path.join(seg_output_dir,  'label')
movie_labeltoggle_output_dir = os.path.join(seg_output_dir,  'togglelabel')
movie_rgb_output_dir = os.path.join(seg_output_dir,  'rgb')    
movie_contour_output_dir = os.path.join(plot_output_dir,  'contour')
movie_quiver_cell_output_dir = os.path.join(plot_output_dir,  'quiver_cell')
movie_quiver_obj_output_dir = os.path.join(plot_output_dir,  'quiver_obj')
movie_label_color_output_dir = os.path.join(plot_output_dir,  'label_color')
movie_rgb_plot_output_dir = os.path.join(plot_output_dir,  'rgb_plot')
movie_panels_plot_output_dir = os.path.join(plot_output_dir,  'panel_plot')
movie_cluster_plot_output_dir = os.path.join(plot_output_dir,  'cluster_plot')

if not os.path.isdir(movie_labeltoggle_output_dir):   os.makedirs(movie_labeltoggle_output_dir)
if not os.path.isdir(movie_mask_output_dir):          os.makedirs(movie_mask_output_dir)
if not os.path.isdir(movie_seg_output_dir):           os.makedirs(movie_seg_output_dir)
if not os.path.isdir(movie_label_output_dir):         os.makedirs(movie_label_output_dir)
if not os.path.isdir(movie_rgb_output_dir):           os.makedirs(movie_rgb_output_dir)
if not os.path.isdir(movie_contour_output_dir):       os.makedirs(movie_contour_output_dir)
if not os.path.isdir(movie_quiver_cell_output_dir):   os.makedirs(movie_quiver_cell_output_dir)
if not os.path.isdir(movie_quiver_obj_output_dir):    os.makedirs(movie_quiver_obj_output_dir)
if not os.path.isdir(movie_label_color_output_dir):   os.makedirs(movie_label_color_output_dir)
if not os.path.isdir(movie_rgb_plot_output_dir):      os.makedirs(movie_rgb_plot_output_dir)
if not os.path.isdir(movie_panels_plot_output_dir):   os.makedirs(movie_panels_plot_output_dir)
# if not os.path.isdir(movie_cluster_plot_output_dir):   os.makedirs(movie_panels_plot_output_dir)
    
         

In [9]:

filenames = [x for x in os.listdir(image_2D_2CH_dir) if os.path.isfile(os.path.join(image_2D_2CH_dir, x)) and ('.tif' in x)]

# if not in run all mode, only test with the first image of the first movie
if not flag_run_all:
    filenames = filenames[3:6]

for filenameID in range(len(filenames)):
    
    filename = filenames[filenameID]
    img = tifffile.imread(os.path.join(image_2D_2CH_dir,filename))
    pax_img = img[1,:,:]
    zyxin_img = img[0,:,:]
       
    # smooth_MIP_pax_img = gaussian_filter(MIP_pax_img,sigma=1,mode='nearest',truncate=3)
    thres_zyxin = 99
    thres_pax = 101

    [new_cell_mask_center, smooth_MIP_pax_img_corrected, smooth_MIP_zyxin_img_corrected] = \
        zyxin_pfak_cell_mask_seg(zyxin_img,pax_img,thres_zyxin, thres_pax, movie_panels_plot_output_dir,filename,flag_run_all,newmap)
    
       

/mnt/d/lding/UbuntuGitCodes/focal_adhesion_clustering/zyxin_pfak_cell_mask_seg.py:109: UserWarning: No contour levels were found within the data range.
  ax[1,2].contour(X, Y, for_orent_distance_taxicab,0,linewidths=0.2,colors='yellow')
/mnt/d/lding/UbuntuGitCodes/focal_adhesion_clustering/zyxin_pfak_cell_mask_seg.py:117: UserWarning: No contour levels were found within the data range.
  ax1.contour(X, Y, for_orent_distance_taxicab,0,linewidths=0.5,colors='yellow')
